In [1]:
# encoding: utf-8
import math
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn import cross_validation  
from sklearn.neighbors import KNeighborsClassifier
 
"""
機器學習做鐵達尼號分析
"""

'\n\xe6\xa9\x9f\xe5\x99\xa8\xe5\xad\xb8\xe7\xbf\x92\xe5\x81\x9a\xe9\x90\xb5\xe9\x81\x94\xe5\xb0\xbc\xe8\x99\x9f\xe5\x88\x86\xe6\x9e\x90\n'

In [3]:
train_data_path = '/Users/macbook/Desktop/code/Simple-Machine-Learning/train.csv'
train_data = pd.read_csv(train_data_path,header=0)

n_train_data = train_data.iloc[:,[0,4]]   #挑想要哪幾欄

#女性存活率
female_tourist = len(train_data[train_data['Sex'] == 'female'])  
female_survived = len(train_data[(train_data['Sex'] == 'female') & (train_data['Survived'] == 1)])  
print female_survived * 1.0 / female_tourist  


0.742038216561


In [4]:
#新增一欄Gender把Sex中女性為1,男性為0
#train_data['Gender'] = train_data['Sex'].map({'female': 1, 'male': 0}).astype(int) 
#astype把它變成int
train_data['Sex'] = train_data['Sex'].map({'female': 1, 'male': 0}).astype(int) 

In [5]:
#把不必要的欄位刪除(Ticket,Name,Cabin,Embarked) 

train_data = train_data.drop(['Ticket','Name','Cabin','Embarked'],axis=1) #axis=0:代表橫軸 axis=1:代表縱軸

#分辨該欄有沒有值，有值為false,無值為True
np.isnan(train_data['Age'])

0      False
1      False
2      False
3      False
4      False
5       True
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17      True
18     False
19      True
20     False
21     False
22     False
23     False
24     False
25     False
26      True
27     False
28      True
29      True
       ...  
861    False
862    False
863     True
864    False
865    False
866    False
867    False
868     True
869    False
870    False
871    False
872    False
873    False
874    False
875    False
876    False
877    False
878     True
879    False
880    False
881    False
882    False
883    False
884    False
885    False
886    False
887    False
888     True
889    False
890    False
Name: Age, dtype: bool

In [6]:
#把Age中的所有數值放入array
m = np.ma.masked_array(train_data['Age'], np.isnan(train_data['Age']))
m

masked_array(data = [22.0 38.0 26.0 35.0 35.0 -- 54.0 2.0 27.0 14.0 4.0 58.0 20.0 39.0 14.0
 55.0 2.0 -- 31.0 -- 35.0 34.0 15.0 28.0 8.0 38.0 -- 19.0 -- -- 40.0 -- --
 66.0 28.0 42.0 -- 21.0 18.0 14.0 40.0 27.0 -- 3.0 19.0 -- -- -- -- 18.0
 7.0 21.0 49.0 29.0 65.0 -- 21.0 28.5 5.0 11.0 22.0 38.0 45.0 4.0 -- --
 29.0 19.0 17.0 26.0 32.0 16.0 21.0 26.0 32.0 25.0 -- -- 0.83 30.0 22.0
 29.0 -- 28.0 17.0 33.0 16.0 -- 23.0 24.0 29.0 20.0 46.0 26.0 59.0 -- 71.0
 23.0 34.0 34.0 28.0 -- 21.0 33.0 37.0 28.0 21.0 -- 38.0 -- 47.0 14.5 22.0
 20.0 17.0 21.0 70.5 29.0 24.0 2.0 21.0 -- 32.5 32.5 54.0 12.0 -- 24.0 --
 45.0 33.0 20.0 47.0 29.0 25.0 23.0 19.0 37.0 16.0 24.0 -- 22.0 24.0 19.0
 18.0 19.0 27.0 9.0 36.5 42.0 51.0 22.0 55.5 40.5 -- 51.0 16.0 30.0 -- --
 44.0 40.0 26.0 17.0 1.0 9.0 -- 45.0 -- 28.0 61.0 4.0 1.0 21.0 56.0 18.0 --
 50.0 30.0 36.0 -- -- 9.0 1.0 4.0 -- -- 45.0 40.0 36.0 32.0 19.0 19.0 3.0
 44.0 58.0 -- 42.0 -- 24.0 28.0 -- 34.0 45.5 18.0 2.0 32.0 26.0 16.0 40.0
 24.0 35.0 22.0 30.0

In [7]:
#用numpy把平均數算出來
mean = np.mean(m).astype(int)
mean

29

In [8]:
"""
    意味著
    for x in train_data['Age']:
        if np.isnan(train_data['Age']) == true: x = mean;
        #else: x = x
"""
train_data['Age'] = train_data['Age'].map(lambda x : mean if np.isnan(x) else x) 

In [9]:
#把每位乘客所有的值array起來
x = train_data.values  
x

array([[   1.    ,    0.    ,    3.    , ...,    1.    ,    0.    ,    7.25  ],
       [   2.    ,    1.    ,    1.    , ...,    1.    ,    0.    ,
          71.2833],
       [   3.    ,    1.    ,    3.    , ...,    0.    ,    0.    ,
           7.925 ],
       ..., 
       [ 889.    ,    0.    ,    3.    , ...,    1.    ,    2.    ,   23.45  ],
       [ 890.    ,    1.    ,    1.    , ...,    0.    ,    0.    ,   30.    ],
       [ 891.    ,    0.    ,    3.    , ...,    0.    ,    0.    ,    7.75  ]])

In [10]:
#把Survived的值array起來
y = train_data['Survived'].values
y

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1,

In [11]:
#把survived那欄拿掉: np.delete(x,第幾欄,axis=1) axis=0:代表橫軸 axis=1:代表縱軸
x = np.delete(x,1,axis=1) 
x

array([[   1.    ,    3.    ,    0.    , ...,    1.    ,    0.    ,    7.25  ],
       [   2.    ,    1.    ,    1.    , ...,    1.    ,    0.    ,
          71.2833],
       [   3.    ,    3.    ,    1.    , ...,    0.    ,    0.    ,
           7.925 ],
       ..., 
       [ 889.    ,    3.    ,    1.    , ...,    1.    ,    2.    ,   23.45  ],
       [ 890.    ,    1.    ,    0.    , ...,    0.    ,    0.    ,   30.    ],
       [ 891.    ,    3.    ,    0.    , ...,    0.    ,    0.    ,    7.75  ]])

In [12]:
#把train_data中一部分拿出來做訓練，再把另一部分拿來測試，再之後要針對每個分類器做出model
#x_train 為已知train_data的feature值, x_test為假設未知的feature
#y_train 為已知train_data的lebal值, y_test為假設未知的lebal
x_train, x_test, y_train, y_test = cross_validation.train_test_split(x,y,test_size=0.3,random_state=1)  

In [13]:
x_train

array([[ 115.    ,    3.    ,    1.    , ...,    0.    ,    0.    ,
          14.4583],
       [ 875.    ,    2.    ,    1.    , ...,    1.    ,    0.    ,   24.    ],
       [  77.    ,    3.    ,    0.    , ...,    0.    ,    0.    ,
           7.8958],
       ..., 
       [  73.    ,    2.    ,    0.    , ...,    0.    ,    0.    ,   73.5   ],
       [ 236.    ,    3.    ,    1.    , ...,    0.    ,    0.    ,    7.55  ],
       [  38.    ,    3.    ,    0.    , ...,    0.    ,    0.    ,    8.05  ]])

In [14]:
x_test

array([[ 863.    ,    1.    ,    1.    , ...,    0.    ,    0.    ,
          25.9292],
       [ 224.    ,    3.    ,    0.    , ...,    0.    ,    0.    ,
           7.8958],
       [  85.    ,    2.    ,    1.    , ...,    0.    ,    0.    ,   10.5   ],
       ..., 
       [ 240.    ,    2.    ,    0.    , ...,    0.    ,    0.    ,
          12.275 ],
       [ 485.    ,    1.    ,    0.    , ...,    1.    ,    0.    ,
          91.0792],
       [  93.    ,    1.    ,    0.    , ...,    1.    ,    0.    ,
          61.175 ]])

In [15]:
#使用decision_tree做訓練model
"""
   運用sklearn的library
"""
decision_tree = tree.DecisionTreeClassifier(max_depth=3)  

In [16]:
#下面是Decision Tree分類器的可改變參數
decision_tree.fit(x_train, y_train) 

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')

In [17]:
decision_tree.score(x_test,y_test)

0.78731343283582089

In [18]:
#使用KNN來做訓練model
"""
運用sklearn的library
"""
knn = KNeighborsClassifier(n_neighbors=25)
knn.fit(x_train, y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=25, p=2, weights='uniform')

In [21]:
#下面是KNN分類器的可改變參數
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=25, p=2, weights='uniform')

In [22]:
knn.score(x_test,y_test)

0.63432835820895528